## ------- How to : build a nnUNet_translation dataset -------

In [1]:
import numpy as np
import nibabel as nib
import shutil, json, glob, os
from tqdm import tqdm 
from concurrent.futures import ThreadPoolExecutor

data1_dir = 'data/mr/'
data2_dir = 'data/mask/'
target_dir = 'data/ct/'

os.environ['nnUNet_results'] = 'results/'
os.environ['nnUNet_raw'] = 'raw/'
os.environ['nnUNet_preprocessed'] = 'preprocessed/'

# example with 2 input modalities
list_datas1 = sorted(glob.glob(os.path.join(data1_dir, '*.nii.gz')))
list_datas2 = sorted(glob.glob(os.path.join(data2_dir, '*.nii.gz')))
list_targets = sorted(glob.glob(os.path.join(target_dir, '*.nii.gz')))

print("input1 ---", len(list_datas1), list_datas1)
print("input2 ---", len(list_datas2), list_datas2)
print("target ---", len(list_targets), list_targets)

input1 --- 5 ['data/mr/1PA070.nii.gz', 'data/mr/1PA073.nii.gz', 'data/mr/1PA074.nii.gz', 'data/mr/1PA076.nii.gz', 'data/mr/1PA079.nii.gz']
input2 --- 5 ['data/mask/1PA070.nii.gz', 'data/mask/1PA073.nii.gz', 'data/mask/1PA074.nii.gz', 'data/mask/1PA076.nii.gz', 'data/mask/1PA079.nii.gz']
target --- 5 ['data/ct/1PA070_0000.nii.gz', 'data/ct/1PA073_0000.nii.gz', 'data/ct/1PA074_0000.nii.gz', 'data/ct/1PA076_0000.nii.gz', 'data/ct/1PA079_0000.nii.gz']


#### Define dataset ID and make paths

In [2]:
dataset_id = 60 # /!\ we will use both the dataset_id and the dataset_id + 1 
dataset_data_name = 'SynthRAD2023_Pelvis_MR_mask'
dataset_target_name = 'SynthRAD2023_Pelvis_CT'

# we will copy the datas
# do not use exist_ok=True, we want an error if the dataset exist already
dataset_data_path = os.path.join(os.environ['nnUNet_raw'], f'Dataset{dataset_id:03d}_{dataset_data_name}') 
os.makedirs(dataset_data_path, exist_ok = True)
os.makedirs(os.path.join(dataset_data_path, 'imagesTr'), exist_ok=True)
os.makedirs(os.path.join(dataset_data_path, 'labelsTr'), exist_ok = True)

dataset_target_path = os.path.join(os.environ['nnUNet_raw'], f'Dataset{dataset_id+1:03d}_{dataset_target_name}') 
os.makedirs(dataset_target_path, exist_ok = True)
os.makedirs(os.path.join(dataset_target_path, 'imagesTr'), exist_ok = True)
os.makedirs(os.path.join(dataset_target_path, 'labelsTr'), exist_ok = True)

#### Copy files and create dummy masks

In [3]:
def process_file(data_path, dataset_path, mat, modality_suffix="_0000"):
    curr_nifti = nib.load(data_path)
    filename = os.path.basename(data_path)
    if not filename.endswith(f'{modality_suffix}.nii.gz'):
        filename = filename.replace('.nii.gz', f'{modality_suffix}.nii.gz')
    curr_nifti.to_filename(os.path.join(dataset_path, f'imagesTr/{filename}'))

    data = curr_nifti.get_fdata()
    # Adjust the mask as needed for your specific use case. By default, the mask is set to 1 for the entire volume.
    # This will be used for foreground preprocessing, cf https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/explanation_normalization.md
    data = np.ones_like(data)

    filename = filename.replace(modality_suffix, '')  # Remove modality suffix for masks
    if not os.path.exists(os.path.join(dataset_path, f'labelsTr/{filename}')):
        nib.Nifti1Image(data, mat).to_filename(os.path.join(dataset_path, f'labelsTr/{filename}')) 

mat = nib.load(list_datas1[-1]).affine

with ThreadPoolExecutor() as executor:
    list(tqdm(executor.map(lambda data_path: process_file(data_path, dataset_data_path, mat, "_0000"), list_datas1), total=len(list_datas1)))

with ThreadPoolExecutor() as executor:
    list(tqdm(executor.map(lambda data_path: process_file(data_path, dataset_data_path, mat, "_0001"), list_datas2), total=len(list_datas2)))

with ThreadPoolExecutor() as executor:
    list(tqdm(executor.map(lambda target_path: process_file(target_path, dataset_target_path, mat), list_targets), total=len(list_targets)))

#### without multithreading
# for data_path in tqdm(list_datas, total=len(list_datas)):
#     process_file(data_path, dataset_data_path, mat)

# for target_path in tqdm(list_targets, total=len(list_targets)):
#     process_file(target_path, dataset_target_path, mat)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:04<00:00,  1.18it/s]


#### Create the dataset.json

In [4]:
# /!\ you will need to edit this with regards to the number of modalities used;
data_dataset_json = {
    "labels": {
        "label_001": "1", 
        "background": 0
    },
    "channel_names": {
        "0": "MR",
        "1": "mask",
        
    },
    "numTraining": len(list_datas1),
    "file_ending": ".nii.gz"
}
dump_data_datasets_path = os.path.join(dataset_data_path, 'dataset.json')
with open(dump_data_datasets_path, 'w') as f:
    json.dump(data_dataset_json, f)

target_dataset_json = {
    "labels": {
        "label_001": "1",
        "background": 0
    },
    "channel_names": {
        "0": "CT",
    },
    "numTraining": len(list_targets),
    "file_ending": ".nii.gz"
}
dump_target_datasets_path = os.path.join(dataset_target_path, 'dataset.json')
with open(dump_target_datasets_path, 'w') as f:
    json.dump(target_dataset_json, f)

#### Apply preprocessing and unpacking 

In [5]:
if 'MPLBACKEND' in os.environ: 
    del os.environ['MPLBACKEND'] # avoid conflicts with matplotlib backend  
    
os.system(f'nnUNetv2_plan_and_preprocess -d {dataset_id} -c 3d_fullres')
os.system(f'nnUNetv2_unpack {dataset_id} 3d_fullres 0')

os.system(f'nnUNetv2_plan_and_preprocess -d {dataset_id + 1} -c 3d_fullres')
os.system(f'nnUNetv2_unpack {dataset_id + 1} 3d_fullres 0')

Fingerprint extraction...
Dataset060_SynthRAD2023_Pelvis_MR_mask
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer


100%|██████████| 5/5 [00:08<00:00,  1.70s/it]


Experiment planning...
Using ZScoreNormalization for image normalization
Using ZScoreNormalization for image normalization
Using ZScoreNormalization for image normalization
Using ZScoreNormalization for image normalization
Attempting to find 3d_lowres config. 
Current spacing: [2.5  1.03 1.03]. 
Current patch size: (np.int64(56), np.int64(160), np.int64(224)). 
Current median shape: [116.         305.82524272 465.04854369]
Using ZScoreNormalization for image normalization
Using ZScoreNormalization for image normalization
Attempting to find 3d_lowres config. 
Current spacing: [2.5    1.0609 1.0609]. 
Current patch size: (np.int64(56), np.int64(160), np.int64(224)). 
Current median shape: [116.         296.91771138 451.50344048]
Using ZScoreNormalization for image normalization
Using ZScoreNormalization for image normalization
Attempting to find 3d_lowres config. 
Current spacing: [2.5      1.092727 1.092727]. 
Current patch size: (np.int64(64), np.int64(128), np.int64(224)). 
Current me

100%|██████████| 5/5 [00:20<00:00,  4.15s/it]


Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

Fingerprint extraction...
Dataset061_SynthRAD2023_Pelvis_CT
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer


100%|██████████| 5/5 [00:06<00:00,  1.27s/it]


Experiment planning...
Using CTNormalization for image normalization
Using CTNormalization for image normalization
Attempting to find 3d_lowres config. 
Current spacing: [2.5  1.03 1.03]. 
Current patch size: (np.int64(56), np.int64(160), np.int64(224)). 
Current median shape: [116.         305.82524272 465.04854369]
Using CTNormalization for image normalization
Attempting to find 3d_lowres config. 
Current spacing: [2.5    1.0609 1.0609]. 
Current patch size: (np.int64(56), np.int64(160), np.int64(224)). 
Current median shape: [116.         296.91771138 451.50344048]
Using CTNormalization for image normalization
Attempting to find 3d_lowres config. 
Current spacing: [2.5      1.092727 1.092727]. 
Current patch size: (np.int64(64), np.int64(128), np.int64(224)). 
Current median shape: [116.         288.2696227  438.35285483]
Using CTNormalization for image normalization
Attempting to find 3d_lowres config. 
Current spacing: [2.5        1.12550881 1.12550881]. 
Current patch size: (np.i

100%|██████████| 5/5 [00:15<00:00,  3.03s/it]


Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################



0

#### Define 2nd modality raw data as gt_segmentations of 1st modality
##### originally used for computing metrics / postprocessing, not sure if needed

In [6]:
nnunet_datas_preprocessed_dir = os.path.join(os.environ['nnUNet_preprocessed'], f'Dataset{dataset_id+1:03d}_{dataset_target_name}') 
nnunet_targets_preprocessed_dir = os.path.join(os.environ['nnUNet_preprocessed'], f'Dataset{dataset_id:03d}_{dataset_data_name}') 

list_targets = glob.glob(os.path.join(f"{dataset_target_path}/imagesTr", '*'))
list_targets.sort()
list_gt_segmentations_datas = glob.glob(os.path.join(f"{nnunet_targets_preprocessed_dir}/gt_segmentations", '*'))
list_gt_segmentations_datas.sort()

print(nnunet_targets_preprocessed_dir)

for (preprocessed_path, gt_path) in zip(list_targets, list_gt_segmentations_datas):
    # here, gt_path is the path to the gt_segmentation in nnUNet_preprocessed.
    print(preprocessed_path, "->", gt_path) # ensure correct file pairing; 
    shutil.copy(src = preprocessed_path, dst = gt_path) # we use shutil.copy to ensure safety, but switching to shutil.move would be more efficient

preprocessed/Dataset060_SynthRAD2023_Pelvis_MR_mask
raw/Dataset061_SynthRAD2023_Pelvis_CT/imagesTr/1PA070_0000.nii.gz -> preprocessed/Dataset060_SynthRAD2023_Pelvis_MR_mask/gt_segmentations/1PA070.nii.gz
raw/Dataset061_SynthRAD2023_Pelvis_CT/imagesTr/1PA073_0000.nii.gz -> preprocessed/Dataset060_SynthRAD2023_Pelvis_MR_mask/gt_segmentations/1PA073.nii.gz
raw/Dataset061_SynthRAD2023_Pelvis_CT/imagesTr/1PA074_0000.nii.gz -> preprocessed/Dataset060_SynthRAD2023_Pelvis_MR_mask/gt_segmentations/1PA074.nii.gz
raw/Dataset061_SynthRAD2023_Pelvis_CT/imagesTr/1PA076_0000.nii.gz -> preprocessed/Dataset060_SynthRAD2023_Pelvis_MR_mask/gt_segmentations/1PA076.nii.gz
raw/Dataset061_SynthRAD2023_Pelvis_CT/imagesTr/1PA079_0000.nii.gz -> preprocessed/Dataset060_SynthRAD2023_Pelvis_MR_mask/gt_segmentations/1PA079.nii.gz


#### Define 2nd modality preprocessed files as ground truth of 1st modality
##### used in training, definitely needed

In [7]:
list_preprocessed_datas_seg_path = sorted(glob.glob(os.path.join(nnunet_targets_preprocessed_dir, 'nnUNetPlans_3d_fullres/*_seg.npy')))

list_preprocessed_targets_path = sorted(glob.glob(os.path.join(nnunet_datas_preprocessed_dir, 'nnUNetPlans_3d_fullres/*.npy')))
list_preprocessed_targets_path = [name for name in list_preprocessed_targets_path if '_seg' not in name]

for (datas_path, targets_path) in zip(list_preprocessed_datas_seg_path, list_preprocessed_targets_path):
    print(targets_path, "->", datas_path)
    shutil.copy(src = targets_path, dst = datas_path) 

preprocessed/Dataset061_SynthRAD2023_Pelvis_CT/nnUNetPlans_3d_fullres/1PA070.npy -> preprocessed/Dataset060_SynthRAD2023_Pelvis_MR_mask/nnUNetPlans_3d_fullres/1PA070_seg.npy
preprocessed/Dataset061_SynthRAD2023_Pelvis_CT/nnUNetPlans_3d_fullres/1PA073.npy -> preprocessed/Dataset060_SynthRAD2023_Pelvis_MR_mask/nnUNetPlans_3d_fullres/1PA073_seg.npy
preprocessed/Dataset061_SynthRAD2023_Pelvis_CT/nnUNetPlans_3d_fullres/1PA074.npy -> preprocessed/Dataset060_SynthRAD2023_Pelvis_MR_mask/nnUNetPlans_3d_fullres/1PA074_seg.npy
preprocessed/Dataset061_SynthRAD2023_Pelvis_CT/nnUNetPlans_3d_fullres/1PA076.npy -> preprocessed/Dataset060_SynthRAD2023_Pelvis_MR_mask/nnUNetPlans_3d_fullres/1PA076_seg.npy
preprocessed/Dataset061_SynthRAD2023_Pelvis_CT/nnUNetPlans_3d_fullres/1PA079.npy -> preprocessed/Dataset060_SynthRAD2023_Pelvis_MR_mask/nnUNetPlans_3d_fullres/1PA079_seg.npy


#### That's it!
You should be able to start training with : 
```
export nnUNet_raw="/data/alonguefosse/nnUNet/raw"
export nnUNet_preprocessed="/data/alonguefosse/nnUNet/preprocessed"
export nnUNet_results="/data/alonguefosse/nnUNet/results"

nnUNetv2_train 50 3d_fullres 0 -tr nnUNetTrainerMRCT
```